In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPLumberLoader, PyMuPDF, PyPDFLoader
from langchain_community.vecotorstores import FAISS
from langchain_chroma import chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub

loader = PDFPLumberLoader()
loader1 = PyMuPDF()
loader2 = PyPDFLoader()

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()
embeddings2 = chroma()

vectorstore = FAISS.from_documents(documents=split_documents, embedding = embeddings)

retriever = vectorstore.as_retriever()

prompt = hub.pull("teddynote/rag-korean")

llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)

chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)
chain.invoke("LG전자가 개발한 생성형 AI 이름은?")

